In [11]:
import pandas as pd

In [12]:
data = pd.read_csv('../../src/ocr_combined.csv')
data.head()

,Unnamed: 0.3,Unnamed: 0,image_path,label,words,bbox,Unnamed: 0.1,Unnamed: 0.2
0,0,0,../../train/catagorized_images/0/10033.tif,0,"['alyeet' 'To!' 'Dae:' 'June' '9,' '1978' 'MOR...","[array([ 49, 112, 87, 122], dtype=int64)\n ar...",NaN,NaN
1,1,1,../../train/catagorized_images/0/10040.tif,0,"['From:' 'Blackmer,' 'Ed' 'To:' 'Whitehurst,' ...","[array([148, 86, 197, 98], dtype=int64)\n ar...",NaN,NaN
2,2,2,../../train/catagorized_images/0/10075.tif,0,"['Kane,' 'Lalsimer,' 'Kane,' 'Sullivanand' 'Ke...","[array([251, 65, 319, 85], dtype=int64)\n ar...",NaN,NaN
3,3,3,../../train/catagorized_images/0/10081.tif,0,"['April' '15,' '1986' 'TO:' 'Ma.' 'M.A.' 'Simm...","[array([120, 142, 168, 154], dtype=int64)\n ar...",NaN,NaN
4,4,4,../../train/catagorized_images/0/10106.tif,0,"['2380' 'Oerard' 'krerus,' 'Pron' 'ew' 'Tork,'...","[array([601, 415, 627, 423], dtype=int64)\n ar...",NaN,NaN


In [13]:
data.drop(columns=['Unnamed: 0.3', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'], inplace=True)

In [14]:
from datasets import Dataset

In [15]:
df = Dataset.from_pandas(data)
df

Dataset({
    features: ['image_path', 'label', 'words', 'bbox'],
    num_rows: 16000
})

In [16]:
data_df = pd.DataFrame.from_dict(df)
print(len(data_df["words"][0]))
print(data_df["words"][0])

1178
['alyeet' 'To!' 'Dae:' 'June' '9,' '1978' 'MORE' 'Tar' 'Control' 'Mr.'
 'E.' 'M.' 'Blackmer' 'From:' 'John' 'L.' 'McKenzie' 'w+' 'Marketing'
 'Tobacco' 'Development' 'has' 'recommended' 'that' 'the' 'tar' 'for'
 'MORE' 'cigarettes,' 'which' 'has' 'trended' 'upward' 'over' 'the' 'last'
 'year' 'or' 'so,' 'could' 'be' 'brought' 'back' 'toward' 'the' 'stated'
 'objective' 'of' '21-22' 'mg.' 'tar' 'by' 'means' 'of' 'a' 'tipping'
 'change.' 'A' 'factory' 'test' 'was' 'made' 'using' 'this' 'change' 'on'
 'MORE' 'Regular.' 'The' 'following' 'analytical' 'and' 'smoke' 'panel'
 'results' 'pertain' 'to' 'that' 'test:' 'Product' 'Code' '25170' '2517B'
 'Description' 'Control' 'Increased' 'Porosity' 'Tipping' 'Nicotine,'
 'mz/cigt.' '1.8' '1.7' '25' '23' 'Tar,' 'mg/ciat.' 'Company-wide' 'Pack'
 'Test' 'Base:' '132' '25170' '25178' 'Preference' '368' '338' 'Milder'
 '348' '31t' 'Overall' 'Better' 'Flavor' '348' '34e' 'This' 'data'
 'indicates' 'that' 'a' '2' 'mg.' 'tar' 'reduction' 'is' 'feasi

In [24]:
from transformers import LayoutLMTokenizer
import torch
import os

In [29]:
tokenizer = LayoutLMTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")

def encode_example(example, max_seq_length=512, pad_token_box=[0, 0, 0, 0]):
	words = example['words']
	normalized_word_boxes = example['bbox']

	# assert len(words) == len(normalized_word_boxes)

	if len(words)  <  len(normalized_word_boxes):
		# os.remove(example['image_path'])
		example['label'] = -1
		return	{}
		
		

	token_boxes = []
	for word, box in zip(words, normalized_word_boxes):
		word_tokens = tokenizer.tokenize(word)
		token_boxes.extend([box] * len(word_tokens))

	# Truncation of token_boxes
	special_tokens_count = 2 
	if len(token_boxes) > max_seq_length - special_tokens_count:
		token_boxes = token_boxes[: (max_seq_length - special_tokens_count)]

	# add bounding boxes of cls + sep tokens
	token_boxes = [[0, 0, 0, 0]] + token_boxes + [[1000, 1000, 1000, 1000]]

	encoding = tokenizer(' '.join(words), padding='max_length', truncation=True)
 
	# Padding of token_boxes up the bounding boxes to the sequence length.
	input_ids = tokenizer(' '.join(words), truncation=True)["input_ids"]
	padding_length = max_seq_length - len(input_ids)
	token_boxes += [pad_token_box] * padding_length
	encoding['bbox'] = token_boxes
	encoding['label'] = example['label']

	assert len(encoding['input_ids']) == max_seq_length
	assert len(encoding['attention_mask']) == max_seq_length
	assert len(encoding['token_type_ids']) == max_seq_length
	assert len(encoding['bbox']) == max_seq_length

	return encoding

In [19]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D

In [30]:
# we need to define the features ourselves as the bbox of LayoutLM are an extra feature
features = Features({
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'label': ClassLabel(names=['refuted', 'entailed']),
    'image_path': Value(dtype='string'),
    'words': Sequence(feature=Value(dtype='string')),
})

encoded_dataset = df.map(lambda example: encode_example(example), 
                                      features=features)

  6%|▌         | 999/16000 [00:00<00:01, 12072.04ex/s]


ArrowInvalid: Could not convert '[' with type str: tried to convert to int64